In [1]:
!pip install networkx 
import networkx as nx

You are using pip version 18.1, however version 19.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
graph = nx.read_gexf("../data/concepnet-best.gexf")
dot = nx.nx_pydot.to_pydot(graph)
dot.set_rankdir("LR")
dot.write_pdf("../data/conceptnet.pdf")

In [3]:
from snn.base import Entity, Relation, SNN

Using TensorFlow backend.


In [4]:
nodes = list(nx.topological_sort(graph))
nodes.reverse()

In [5]:
entities = {}

for n in nodes:
    parents = list(graph.neighbors(n))
    parents = {entities[p] for p in parents}
    entity = Entity(n, *parents)
    entities[n] = entity

In [6]:
import pandas as pd
import tqdm
import json

In [7]:
snn_relations = []

for i, (src, dst, r) in enumerate(json.load(open("../data/concepnet-relations.json"))):
    r = r.split("/")[-1]
    snn_relations.append(Relation(label="%d-%s" % (i,r), src=entities[src], dst=entities[dst]))
    
snn_relations

[0-Synonym(<vermin> , <insect>),
 1-Synonym(<unit> , <whole>),
 2-Synonym(<human_activity> , <act>),
 3-Synonym(<thing> , <entity>),
 4-Causes(<change> , <change>),
 5-Antonym(<animal> , <mineral>),
 6-Desires(<person> , <communication>),
 7-Synonym(<region> , <state>),
 8-Antonym(<vertebrate> , <invertebrate>),
 9-UsedFor(<writing> , <information>),
 10-InstanceOf(<film> , <film>),
 11-DerivedFrom(<relationship> , <relation>),
 12-Synonym(<action> , <process>),
 13-UsedFor(<animal> , <food>),
 14-Antonym(<work> , <being>),
 15-Synonym(<action> , <performing>),
 16-UsedFor(<system> , <organization>),
 17-HasContext(<practice> , <medicine>),
 18-SimilarTo(<relationship> , <relation>),
 19-Synonym(<act> , <work>),
 20-DerivedFrom(<worker> , <work>),
 21-PartOf(<body_part> , <organism>),
 22-Synonym(<action> , <movement>),
 23-Synonym(<region> , <area>),
 24-Synonym(<message> , <communication>),
 25-PartOf(<genus> , <family>),
 26-Synonym(<situation> , <position>),
 27-PartOf(<food> , <fo

In [8]:
full_conceptnet = nx.read_gexf("../data/conceptnet.gexf")

In [ ]:
concepts = set(nodes)

for node in tqdm.tqdm(full_conceptnet.nodes):
    descendants = list(nx.descendants(full_conceptnet, node))
    for des in descendants:
        if des in concepts:
            full_conceptnet.add_edge(node, des)
        elif full_conceptnet.has_edge(node, des):
            full_conceptnet.remove_edge(node, des)

  3%|▎         | 5080/171188 [01:12<41:22, 66.90it/s]

In [65]:
for node in concepts:
    full_conceptnet.add_edge("/c/en/%s" % node, "SUPER")

def is_concept_instance(node):
    if node == "SUPER":
        return False
    
    if not node.startswith("/c/"):
        node = "/c/en/%s" % node
            
    if node.split("/")[3] in concepts:
        return False
    
    return nx.has_path(full_conceptnet, node, "SUPER")

In [75]:
from tqdm import tqdm_notebook as tqdm

instances = []

for n in full_conceptnet.nodes:
    if is_concept_instance(n):
        instances.append(n)

In [77]:
len(instances)

134025

In [14]:
from keras.layers import Input

In [79]:
x = Input(shape=(len(instances),))
snn = SNN(entities=entities.values(), relations=[])
y = snn(x)
model = snn.build()